## Wczytywanie danych
Paczka Sklearn (Scikit Learn) to kolejne bardzo popularne narzędzie do uczenia maszynowego. Posiada bardzo przejrzyste API i spore wsparcie (scikit-learn.org/). W dzisiejszych zadaniach skupimy się na ładowaniu zbiorów danych, ich transformacji oraz algorytmach klasyfikacji. Podczas dzisiejszych laboratoriów wykorzystamy:
<ul>
    <li>NLTK - udostępniające metody prostego przetwarzania tekstu (tokenizacja, lematyzacja, stemming)</li>
    <li>Sklearn - paczkę do uczenia maszynowego</li>
    <li>Pandas - bibliotekę do wczytywania i obsługi zbiorów danych</li>
</ul>
<span style="color: #ff0000">Ponieważ część kodu jest już stworzona, w każdym zadaniu wyszczególnione są numery linii, w których należy wprowadzić modyfikacje, aby rozwiązać zadanie. Jeśli nie widzisz numeracji linii w kodzie w otwartym notebooku - możesz włączyć tę funkcjonalność poprzez wybór View -> toggle line numbers w górnym menu.</span><br/><br/>
Najpierw wczytajmy dane tekstowe ze zbioru, w którym posiadamy zestaw wiadomości e-mail oznaczonych jako spamowe lub niespamowe.
Ponieważ będziemy rozwiązywać problem klasyfikacji, oddzielamy dane do trenowania klasyfikatora oraz do weryfikacji jego jakości.
<br/>

<strong>Przeanalizuj i uruchom poniższy fragment kodu.</strong> Załaduje on odpowiednie dane do dwóch obiektów:
<ol>
<li>train: zbiór treningowy - dokumenty na których nauczymy klasyfikator</li>
<li>test: zbiór testowy - dokumenty na których przetestujemy klasyfikator</li>
</ol>

In [1]:
import pandas
import numpy as np

# --- Ładowanie danych i oddzielanie zbioru treningowego od testowego ---
import scipy.sparse
full_dataset = pandas.read_csv('resources/spam-emails.csv', encoding='utf-8')

# ponieważ nazwy kategorii zapisane są z użyciem stringów: "ham"/"spam",
# wykonujemy mapowanie tych wartości na liczby, co będzie potrzebne do wykonania klasyfikacji.
full_dataset['label_num'] = full_dataset.label.map({'ham': 0, 'spam': 1})

# ustaw seed na 0, aby zapewnić powtarzalność eksperymentu
np.random.seed(0)
train_indices = np.random.rand(len(full_dataset)) < 0.7
train: pandas.DataFrame = full_dataset[train_indices]
test: pandas.DataFrame = full_dataset[~train_indices]

# --- Wyświetlanie statystyk ---
print(f"Elementów w zbiorze treningowym/testowym: {len(train)}/{len(test)}")
print(f"Liczność klas w zbiorze treningowym/testowym: {train.label.value_counts()}/{test.label.value_counts()}")

# wyświetl próbkę danych
full_dataset.head()

Elementów w zbiorze treningowym/testowym: 1624/733
Liczność klas w zbiorze treningowym/testowym: ham     1111
spam     513
Name: label, dtype: int64/ham     517
spam    216
Name: label, dtype: int64


,label,text,label_num
0,ham,Re: What to choose for Core i5 64 bits?>>> If ...,0
1,spam,"Strictly Private.Gooday, With warm heart my fr...",1
2,ham,"Re: Flash is open?On Sat, 15 May 2010 00:27:32...",0
3,ham,Re: Alsa/Redhat 8 compatabilityMatthias Saou (...,0
4,spam,"Hey hibody, Save 80% today Lixi Eights followi...",1


## Transformacja danych
Aby zastosować większość algorytmów uczenia maszynowego - dane wejściowe muszą być reprezentowane jako wektory liczb. Wykorzystajmy zatem narzędzia dostarczone przez Scikit-learn do tego celu. Użyjmy klasy **CountVectorizer()**, aby podzielić poszczególne dokumenty na słowa, a następnie stworzyć reprezentację "bag of words"
Dla przypomnienia - "bag of words" tworzony jest w nastepujący sposób:
<ol>
<li>Przeglądamy wszystkie dostępne dokumenty i tworzymy listę wszystkich unikalnych słów jakie napotkaliśmy (słownik).</li>
<li>Stworzona lista wyznacza nam wektor cech - każda pozycja w takim wektorze oznacza jedno z napotkanych słów.</li>
<li>Każdy z dokumentów mapowany jest na wektor cech poprzez zapisanie ile razy każde ze słów dokumentu wystąpiło w nim.</li>
</ol>
Przykład wektoryzacji znajduje się poniżej:

In [6]:
# PRZYKŁAD ------------------------------------------
# np. Dla dwóch dokumentów:
# Dokument 1: Ala ma kota i ma psa 
# Dokument 2: Kot ma Alę

# Poszczególne kroki wyglądają następująco:
# Lista unikalnych słów:                 [Ala, ma, kota, i, psa, Kot, Alę] 
# Szablon wektora cech:                  [  0,  0,    0, 0,   0,   0,   0] - wektor jest tyluelementowy, ile mamy  unikalnych słów 
# Osadzenie dokumentu 1 jako wektor cech: [  1,  2,    1, 1,   1,   0,   0] - słowo "ma" pojawia się w dok. 2 razy, "kot" i "Alę" - wcale
# Analogicznie dokument 2:               [  0,  1,    0, 0,   0,   1,   1]

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
# Stwórz macierz liczbową z danych.
# W wierszach mamy kolejne dokumenty,
# w kolumnach kolejne pola wektora cech odpowiadające unikalnym słowom (bag of words)
X_train_counts = vectorizer.fit_transform(train['text'])
# analogicznie jak wyżej — dla zbioru testowego.
X_test_counts = vectorizer.transform(test['text'])

# Wyświetl rozmiar macierzy.
# Pierwsze pole — liczba dokumentów,
# drugie — liczba cech (stała dla wszystkich dokumentów)
print(f"Rozmiar stworzonej macierzy: {X_train_counts.shape}")
print(f"Liczba dokumentów: {X_train_counts.shape[0]}")
print(f"Rozmiar wektora bag-of-words: {X_train_counts.shape[1]}")


Rozmiar stworzonej macierzy: (1624, 37325)
Liczba dokumentów: 1624
Rozmiar wektora bag-of-words: 37325


### Uwaga:
Na zbiorze treningowym użyto funkcji - fit_transform(), na testowym - transform(). <br/>
**Dlaczego?** fit_transform() wykonuje dwie operacje - tworzy i zapisuje listę wszystkich unikalnych słów (słownik) oraz zamienia dokument na wektor o długości takiej jak słownik. transform() natomiast wykorzystuje istniejący już słownik i wykonuje z jego użyciem transformację do wektora. 
<br/>
Ponieważ zbiór treningowy jest zazwyczaj liczniejszy - z reguły znajdziemy w nim więcej różnych słów. Ponadto, wszystkie słowa, które mogą pomóc w klasyfikacji i tak muszą znaleźć się w zbiorze treningowym aby móc się nauczyć ich wykorzystania. Nie nadpisuje się zatem słownika za pomocą zbioru testowego i tworzy się go tylko raz - podczas treningu, wykorzystując go następnie do tworzenia nowych wektorów z nieobserwowanych podczas treningu dokumentów.

# Zadanie 1 (1 punkt):
Jak się pewnie domyślasz - reprezentacja bag-of-words będzie miała bardzo wiele zer w wygenerowanych macierzach (macierzach, w których w poszczególnych wierszach będziemy mieli poszczególne dokumenty, a w kolumnach wektory słów reprezentacji bag of words). Rozmiar macierzy z poprzedniego listingu pokazuje, że każdy dokument opisany jest wektorem 37325 pozycji, ponieważ tyle różnych słów zostało wykrytych po analizie wszystkich dokumentów treningowych. Większość dokumentów analizowanych osobbno zawierać będzie pewnie co najwyżej kilkadziesiąt/kilkaset różnych słów.

***Zadanie: Napisz fragment kodu, który zliczy:***
<ol>
    <li><strong>jaki procent macierzy X_train_counts ma elementy o wartości różnej od zera</strong></li>
    <li><strong>ile tokenów (łącznie, nie tylko unikalne) występuje w macierzy X_train_counts?</strong></li>
</ol>
Wskazówka - ponieważ zer w tej macierzy jest istotnie dużo - dane po transformacji CountVectorizerem trzymane są w specjalnym formacie, w którym zapisuje się tylko elementy mające wartości różne od zera, w tzw. macierzy rzadkiej (sparse matrix). Aby przeiterować po takiej macierzy, można wykorzystać następujące fragmenty kodu:

<strong>cx = X_train_counts.tocoo()</strong> - transformuj macierz do reprezentacji koordynatowej (patrz niżej) <br/>
<strong>for doc_id, word_id, count in zip(cx.row, cx.col, cx.data):</strong> - pozwala ona na iterowanie po wszystkich niezerowych elementach, w każdym kroku otrzymując 3 zmienne - numer wiersza (numer dokumentu), numer kolumny (identyfikator słowa ze słownika) oraz licznik mówiący ile razy dane słowo wystąpiło w danym dokumencie. <span style="color: #ff0000">(Do wykonania zadania musisz zaktualizować linijki 3, 7 i 8)</span>

In [3]:
# tu zapisz liczbę wszystkich tokenów w macierzy
count_tokens = 0
# tu zapisz ilość elementów niezerowych w macierzy
count_nonzero = 0
# tu zapisz ilość komórek w macierzy (ilość wierszy * ilość kolumn, rozważ użycie pola 'shape' na macierzy X_train_counts)
count_all = np.multiply(*X_train_counts.shape)

#iteracja po elementach niezerowych
cx = X_train_counts.tocoo()
for (doc_id, word_id, count) in zip(cx.row, cx.col, cx.data):
  count_tokens += count
  count_nonzero += count > 0

print(f"W datasecie znajduje się: {count_tokens} tokenów.")
print(f"Macierz posiada {100.0 * count_nonzero / count_all:.3f}% niezerowych elementów.")

W datasecie znajduje się: 435292 tokenów.
Macierz posiada 0.394% niezerowych elementów.


<div class="alert alert-block alert-success">
    <strong>Oczekiwany rezultat:</strong> <br/>
Mniej niż 1% elementów niezerowych (!) <br/>
Ponad 400000 tokenów
</div>

# Zadanie 2 (1 punkt) - słowa charakteryzujące klasy

Wykorzystajmy macierz X_train_counts wykorzystywaną w poprzednim zadaniu, a także etykiety kategorii, aby stworzyć listy najczęściej występuących słów w danych kategoriach. <br/><br/>
Aby ułatwić zadanie, utworzono większość funkcji **get_top_occuring_words()** tworzącej taki ranking<br/>
**Zadanie 2a (0.5 punktu)**: twoim zadaniem jest zaktualizowanie wartości pola: **category_word_counts[category][word]**, tak, aby poprawnie zliczyć ile razy dane słowo wystąpiło w kategorii. <span style="color: #ff0000">(zaktualizuj linijkę 20)</span>
<br/>
Czy najczęstsze słowa pozwalają rozdzielić kategorie SPAM od HAM?  

In [9]:
from collections import defaultdict

def get_top_occurring_words(X_train_counts, how_many_words, vectorizer, train):
  # stwórz mapowanie pozycji wektora bag-of-words na konkretne słowa
  id_to_word = {v: k for k, v in vectorizer.vocabulary_.items()}
  cx = X_train_counts.tocoo()

  # słownik, w którym przeprowadzimy zliczanie
  category_word_counts = defaultdict(lambda: defaultdict(int))
  for (doc_id, word_id, count) in zip(cx.row, cx.col, cx.data):
    category = train.iloc[doc_id]['label']
    word = id_to_word[word_id]
    category_word_counts[category][word] += count

  # wyświetl nazwy kategorii i n-najczęściej występujących w nich słów
  for category_name in category_word_counts:
    # posortowany dict() słowo -> liczność, wg liczności, malejąco
    sorted_cat = sorted(category_word_counts[category_name], key=category_word_counts[category_name].get, reverse=True)
    # wyświetl nazwę kategorii i top n słów
    print(f"{category_name}: {sorted_cat[:how_many_words]}")

# wywołanie funkcji
get_top_occurring_words(X_train_counts, 12, vectorizer, train)

ham: ['the', 'to', 'of', 'and', 'is', 'in', 'it', 'that', 'for', 'you', 'on', 'with']
spam: ['the', 'of', 'to', 'and', 'in', 'you', 'nbsp', 'for', 'is', 'your', 'this', 'as']


### Wektoryzacja Tf-Idf

Po wykonaniu zadania 2a widzimy, że najczęściej występujące słowa w każdej kategorii mają niewielką użytezczność (pasują do każdej kategorii). Aby sprawić, żeby na czele rankingu znalazły się słowa charakterystyczne dla danej klasy, możemy użyć metody Tf-idf. <br/>
**Zadanie 2b:
Nadpisz wartości X_train_counts oraz X_test_counts wykorzystując w tym celu TfidfVectorizer** (http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) zamiast CountVectorizer, ustaw parametr max_df na 0.4 (tzn. ignoruj słowa, które występują w więcej niż 40% dokumentów). Następnie wykonaj stworzoną w zadaniu 2 funkcję get_top_occuring_words(), aby sprawdzić, czy ranking najważniejszych słów się zmienił. Czy zmieniony zestaw słów lepiej reprezentuje kategorie? <span style="color: #ff0000">(zaktualizuj linie 3, 4, 5)</span>

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.4)
# Stwórz macierz wektorów. W wierszach mamy kolejne dokumenty, w kolumnach kolejne pola wektora cech odpowiadające unikalnym słowom
X_train_counts: scipy.sparse.csr_matrix = vectorizer.fit_transform(train['text'])
# analogicznie dla zbioru testowego.
X_test_counts: scipy.sparse.csr_matrix = vectorizer.transform(test['text'])

# wywołanie funkcji
get_top_occurring_words(X_train_counts, 12, vectorizer, train)

ham: ['debian', 'org', 'lists', '20', 'unsubscribe', 'linux', 'net', 'wrote', 'list', 'my', 'can', 'www']
spam: ['nbsp', 'our', '20', 'click', 'here', 'spam', '2009', 'content', 'hibody', 'we', 'free', 'all']


# Zadanie 3 - Stemming i lematyzacja (1 punkt)
Często istotne słowa występują w wielu odmianach (szczególnie w językach fleksyjnych, takich jak nasz), np: university - universities ; pay - paid - paying - pays . Wielość odmian słów ma swoje przełożenie na rozmiar słownika.
<br/><br/>
W niektórych warunkach, w szczególności:
<ul>
<li>Kiedy mamy ograniczoną pamięć</li>
<li>Kiedy ważny jest dla nas czas działania algorytmu</li>
<li>Kiedy istnieje ryzyko przeuczenia</li>
</ul>
warto rozważyć znormalizowanie słów, tak, aby zmniejszyć rozmiar słownika, a co za tym idzie wymagania pamięciowe (a co za tym idzie - czas treningu/klasyfikacji). Ograniczenie rozmiaru słownika może też zapobiec przeuczeniu. Normalizację możemy wykonać np. poprzez zastosowanie stemmingu lub lematyzacji dla poszczególnych wyrazów.
<br/>
<strong>Zadanie 3a (0.5 punktu)</strong>: Z użyciem biblioteki NLTK wykonaj zarówno lematyzację (używając WordNetLemmatizer) jak i stemming (używając PorterStemmer) tekstu zawartego w sample_text. Uwaga - lematyzator opcjonalnie wymaga pos-tagu dla tokenu. Przekaż do funkcji lematyzującej zmienną current_word_postag jako drugi argument. <span style="color: #ff0000">(zaktualizuj linie 20, 21, 34, 35)</span>

<strong>Zadanie 3b (0.5 punktu)</strong>: O ile zmniejszyła się liczba unikalnych słów po zastosowaniu lematyzacji? Odpowiedź zawrzyj w komentarzu. <span style="color: #ff0000">(linijki 43:45)</span>

In [65]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk

# Lematyzator wymaga, aby dla danego słowa podać mu, czy jest to czasownik, rzeczownik czy inny POS-tag.
# Funkcja jest adapterem tagów nadanych przez funkcję pos_tag do tagów wymaganych przez lematyzator
def get_wordnet_pos(treebank_tag):
  match treebank_tag:
    case tag if tag.startswith('J'):
      return wordnet.ADJ
    case tag if tag.startswith('V'):
      return wordnet.VERB
    case tag if tag.startswith('R'):
      return wordnet.ADV
    case _:
      return wordnet.NOUN

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

sample_text = "There are some cheaper alternatives for buying the red trousers. There is a discount, it is so cheap!"

# dzielimy tekst na słowa
tokenized = word_tokenize(sample_text)
# nadajemy pos-tagi (rzeczownik, czasownik przymiotnik...) każdemu słowu
pos_tokens = nltk.pos_tag(tokenized)

# Zlematyzowane słowa
lemmatized = [lemmatizer.lemmatize(token, get_wordnet_pos(tag))
              for (token, tag) in nltk.pos_tag(tokenized)]
# Wystemowane słowa
stemmed = [stemmer.stem(token) for (token, _) in nltk.pos_tag(tokenized)]

print(f"{'Bazowy tekst:':<21}\n\t{sample_text}\n")
print(f"{'Wystemowany tekst:':<21}\n\t{' '.join(stemmed)}\n")
print(f"{'Zlematyzowany tekst:':<21}\n\t{' '.join(lemmatized)}\n")

# Ile unikalnych tokenów znajduje się w tekście bazowym?:
print(f"{'Liczba unikalnych tokenów:':<27}{len(set(tokenized))}")
# Ile unikalnych tokenów znajduje się w tekście wystemowanym?:
print(f"{'Liczba unikalnych stemów:':<27}{len(set(stemmed))}")
# Ile unikalnych tokenów w tekście zlematyzowanym?:
print(f"{'Liczba unikalnych lematów:':<27}{len(set(lemmatized))}")

# Różnica ilości unikalnych tokenów między tekstem bazowym a wystemowanym:
print(f"{'Różnica między liczbą unikalnych tokenów i stemów:':<52}{abs(len(set(tokenized)) - len(set(stemmed)))}")
# Różnica ilości unikalnych tokenów między tekstem bazowym a zlematyzowanym:
print(f"{'Różnica między liczbą unikalnych tokenów i lematów:':<52}{abs(len(set(tokenized)) - len(set(lemmatized)))}")

Bazowy tekst:        
	There are some cheaper alternatives for buying the red trousers. There is a discount, it is so cheap!

Wystemowany tekst:   
	there are some cheaper altern for buy the red trouser . there is a discount , it is so cheap !

Zlematyzowany tekst: 
	There be some cheap alternative for buy the red trouser . There be a discount , it be so cheap !

Liczba unikalnych tokenów: 19
Liczba unikalnych stemów:  19
Liczba unikalnych lematów: 17
Różnica między liczbą unikalnych tokenów i stemów:  0
Różnica między liczbą unikalnych tokenów i lematów: 2


---
# Zadanie 4 (1 punkt) - klasyfikacja i interpretacja wyników
Mając już dobrą reprezentację danych i wiedząc jak działa normalizacja - możemy klasyfikować! <br/>
Istnieje wiele algorytmów, które dobrze radzą sobie z klasyfikacją tekstu, kilka przykładów to: 
<ul>
<li>Naiwny klasyfikator Bayesa</li>
<li>Maszyna wektorów nośnych - SVM</li>
<li>Sieci neuronowe</li>
</ul>
O sieciach neuronowych więcej powiemy na jednych z przyszłych laboratoriów. <br/>
<strong>Zadanie 4a (0.5 punktu)</strong> Wykorzystując przetworzoną postać danych: X_train_counts, X_test_counts z poprzednich zadań oraz dokumentację sklearn, zaimplementuj klasyfikację z użyciem naiwnego klasyfikatora Bayesa (MultinomialNB). <span style="color: #ff0000">(zaktualizuj linie 7, 9, 12)</span>

In [126]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Funkcja pomocnicza zamieniająca identyfikatory numeryczne na tekstowe
def labels_as_strings(vector_of_indices):
  return ['ham' if ind == 0 else 'spam' for ind in vector_of_indices]

# Stwórz klasyfikator
nb = MultinomialNB()

# Wytrenuj klasyfikatorn
y = X_train_counts
nb.fit(X_train_counts, train['label_num'])

print("Ile elementów testowych udało się poprawnie zaklasyfikować?")
results: np.ndarray = nb.predict(X_test_counts)
accuracy = len(test[results == test['label_num']])

print(f"Poprawnie zaklasyfikowano: {100 * accuracy / len(results):.3f}% {accuracy}/{len(results)} elementów")

# Testowanie klasyfikatora — szerokie podsumowanie uwzględniające miary:
# - precision
# - recall
# - f1
print("Szczegółowy raport (per klasa)")
print(classification_report(
  labels_as_strings(test['label_num']),
  labels_as_strings(nb.predict(X_test_counts)))
)

Ile elementów testowych udało się poprawnie zaklasyfikować?
Poprawnie zaklasyfikowano: 88.404% 648/733 elementów
Szczegółowy raport (per klasa)
              precision    recall  f1-score   support

         ham       0.86      1.00      0.92       517
        spam       1.00      0.61      0.76       216

    accuracy                           0.88       733
   macro avg       0.93      0.80      0.84       733
weighted avg       0.90      0.88      0.87       733



**Zadanie 4b (0.5 punktu)
Po analizie szczegółowego raportu z zadania 4a - odpowiedz na poniższe pytania i zapisz odpowiedzi w komentarzu:**
<ol>
<li>Która miara mówi nam o tym, jak wiele spośród elementów uznanych za spam rzeczywiście jest spamem?</li>
<li>Która miara mówi nam o tym, jak wiele spośród wszystkich elementów rzeczywiście będących spamem zostało wykrytych jako spam?</li>
<li>Która kategoria została w ogólnym rozrachunku lepiej rozpoznana przez klasyfikator, jeśli zależy nam bardziej na tym, żeby klasyfikator, jeśli mówi, że coś należy do danej klasy, raczej się w tym nie mylił, niż żeby wykrył wszystkie elementy klasy?</li>
</ol>

In [197]:
# odp zad 4.1: precision
# odp zad 4.2: recall
# odp zad 4.3: spam

Sklearn jest bardzo wdzięcznym narzędziem, w którym proces klasyfikacji możemy wykonać w zaledwie kilku linijkach. Bardzo przydatną klasą jest klasa Pipeline, która definiuje sekwencję kroków, które wykonujemy wywołując metodę fit().
W naszym przypadku mamy dwa kroki:
<ol>
    <li>Wektoryzacja - zamienia dane zapisane w postaci tekstowej na macierz z wektorami bag-of-words.</li>
    <li>Klasyfikacja - wytrenowanie klasyfikatora.</li>
</ol>
W zdefiniowanym obiekcie typu pipeline, i+1 element pipeline'u na wejściu dostaje dane z wyjścia i-tego elementu (Zatem nasz klasyfikator otrzyma dane przetworzone przez TfidfVectorizer). <br/>
Metoda fit na wejściu przyjmuje listę dokumentów w formie tekstowej, oraz oczekiwane etykiety w formie liczbowej.
<br/>
Analogicznie w procesie klasyfikowania nowych tekstów z użyciem istniejącego modelu - metoda predict() wykona sekwencję kroków: wektoryzacja + klasyfikacja dla zadanej listy surowych tekstów). <br/>
Zapoznaj się z poniższym kodem i uruchom go.

In [127]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas
import numpy as np

# ------------------- WCZYTANIE DANYCH -----------

# Wczytaj dane z pliku CSV
full_dataset = pandas.read_csv('resources/spam-emails.csv', encoding='utf-8')
# Ponieważ nazwy kategorii zapisane są z użyciem stringów: "ham"/"spam",
# Wykonujemy mapowanie tych wartości na liczby, aby móc wykonać klasyfikację.
full_dataset['label_num'] = full_dataset.label.map({'ham': 0, 'spam': 1})

# Ustaw seed na 0, aby zapewnić powtarzalność eksperymentu
np.random.seed(0)
# Wylosuj 70% wierszy, które znajdą się w zbiorze treningowym
train_indices = np.random.rand(len(full_dataset)) < 0.7

# Wybierz zbiór treningowy (70%)
train: pandas.DataFrame = full_dataset[train_indices]
# Wybierz zbiór testowy (dopełnienie treningowego - 30%)
test: pandas.DataFrame = full_dataset[~train_indices]

# ------------------- STWORZENIE PIPELINE'U -----------

# Stwórzmy pipeline surowy tekst -> TFIDF vectorizer -> klasyfikator
pipeline = Pipeline([
  ('tfidf', TfidfVectorizer(max_df=0.4)),
  ('clf', MultinomialNB()),
])

# ------------------- TRANSFORMACJA I UCZENIE -----------

# Zwektoryzujmy dane i wytrenujmy klasyfikator na zbiorze treningowym
pipeline.fit(train['text'], train['label_num'])

# ------------------- KLASYFIKACJA PRZYKŁADOWEGO TEKSTU -----------

text_to_predict = "NEED TO FIND SOMETHING? ::FREE MORTGAGE QUOTE:: To be removed from this list, click here. "
if pipeline.predict([text_to_predict]) == 1:
  detected = 'SPAM'
else:
  detected = 'HAM'
print(f"Tekst {text_to_predict}, zaklasyfikowany został jako: {detected}")

# ------------------- OCENA KLASYFIKATORA -----------
accuracy = 100 * pipeline.score(test['text'], test['label_num'])
print(f"W zbiorze testowym {accuracy:.3f}% przypadków zostało poprawnie zaklasyfikowanych!")

Tekst NEED TO FIND SOMETHING? ::FREE MORTGAGE QUOTE:: To be removed from this list, click here. , zaklasyfikowany został jako: SPAM
W zbiorze testowym 88.404% przypadków zostało poprawnie zaklasyfikowanych!


# Zadanie 5 (1 punkt): dobór parametrów klasyfikacji
Poniżej znajduje się kod tworzący pipeline składający się z dwóch elementów: TfidfVectorizera oraz klasyfikatora naiwnego Bayesa - MultinomialNB. Wektoryzator tworzy model bag-of-words, który uwzględnia jedynie 1000 najważniejszych słów w słowniku. W celu zastosowania stemmingu oraz lematyzatora w treningu i predykcji stoworzona została klasa TheTokenizer, która poza podziałem tekstu na słowa wykonuje również zadania normalizacji wg. ustalonych flag: **use_stemming, use_lemmatization, use_stopword_removal**. <br/>
<strong>Zadanie 5a (0.5 punktu)</strong>: <br/>
Zweryfikuj jak zmiana wartości flag **use_stemming, use_lemmatization, use_stopword_removal**, a co za tym idzie wykorzystanie lamatyzacji, stemmingu i usuwania najczęstszych słow wpływa na miary precision, recall i f1 stworzonego klasyfikatora. Wyniki zapisz w komentarzu. <span style="color: #ff0000">(modyfikuj linie 16, 17, 18, komentarz - w kolejnej komórce)</span><br/>
<strong> Zadanie 5b (0.5 punktu)</strong>: <br/>
Ustaw flagi **use_stemming, use_lemmatization, use_stopword_removal** z linii 16,17 i 18 na False, i porównaj wartości precision recall i f1 dla klasyfikatora, ktory wykorzystuje CountVectorizer i takiego, który wykorzystuje TfidfVectorizer. Pozostaw parametr max_features=1000 w obu przypadkach. Który wektoryzator jest lepszy? <span style="color: #ff0000">(modyfikuj linię 84)</span>
</ol>

In [176]:
from typing import Iterable
from sklearn.datasets import fetch_20newsgroups
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from nltk.corpus import stopwords
from dataclasses import dataclass

@dataclass
class TheTokenizer(object):
  use_stemming: bool = False
  use_lemmatization: bool = False
  use_stopword_removal: bool = False

  lemmatizer = WordNetLemmatizer()
  stemmer = PorterStemmer()
  stopwords = set(stopwords.words('english'))

  def __call__(self, doc):
    if not self.use_stemming and not self.use_lemmatization: # tokenizuj i ew. lematyzuj/stemuj/usuń stopwords w zależności od ustawionych flag
        return [t for t in word_tokenize(doc) if self.is_allowed(t)]
    elif self.use_stemming and not self.use_lemmatization:
        return [self.stem_token(t) for t in word_tokenize(doc) if self.is_allowed(t)]
    elif self.use_lemmatization and not self.use_stemming:
        return [self.lemmatize_token(t, pos) for t, pos in pos_tag(word_tokenize(doc)) if self.is_allowed(t)]

  def stem_token(self, t):
    return self.stemmer.stem(t)

  def lemmatize_token(self, t, postag):
    return self.lemmatizer.lemmatize(t, self.adapt_token_to_wordnet(postag))

  def is_allowed(self, token: str):
    return not self.use_stopword_removal or not token in self.stopwords

  def adapt_token_to_wordnet(self, treebank_tag: str) -> str:
    match treebank_tag:
      case tag if tag.startswith('J'):
        return wordnet.ADJ
      case tag if tag.startswith('V'):
        return wordnet.VERB
      case tag if tag.startswith('R'):
        return wordnet.ADV
      case _:
        return wordnet.NOUN

print("Pobieranie danych...")
categories = [
  'misc.forsale',
  'soc.religion.christian',
  'sci.space',
  'talk.politics.guns',
  'comp.graphics',
  'sci.med',
  'rec.motorcycles',
  'sci.med',
  'sci.electronics',
  'talk.politics.misc',
  'comp.sys.mac.hardware'
]
dataset_train = fetch_20newsgroups(subset='train', categories=categories)
dataset_test = fetch_20newsgroups(subset='test', categories=categories)
print("Pobieranie danych zakończone.")

Pobieranie danych...
Pobieranie danych zakończone.


Część A:

In [179]:
print("Tworzenie pipeline'u - TfidfVectorizer + stemming")
pipeline = Pipeline([
  ('vectorizer', TfidfVectorizer(tokenizer=TheTokenizer(use_stemming=True), max_features=1000)),
  ('clf', MultinomialNB()),
])

print("Uczenie pipeline'u")
pipeline.fit(dataset_train.data, dataset_train.target)

print("Ile różnych słów tworzy wektor bag of words (jaki jest rozmiar słownika)")
print(f"W słowniku znajduje się {len(pipeline.named_steps['vectorizer'].vocabulary_.keys())} różnych słów")

print("Ocena klasyfikatora TfidfVectorizer + stemming:")
print(
  classification_report(
    labels_as_strings(dataset_test.target),
    labels_as_strings(pipeline.predict(dataset_test.data)))
)

Tworzenie pipeline'u - TfidfVectorizer + stemming
Uczenie pipeline'u
Ile różnych słów tworzy wektor bag of words (jaki jest rozmiar słownika)
W słowniku znajduje się 1000 różnych słów
Ocena klasyfikatora TfidfVectorizer + stemming:
                        precision    recall  f1-score   support

         comp.graphics       0.69      0.78      0.73       389
 comp.sys.mac.hardware       0.79      0.77      0.78       385
          misc.forsale       0.76      0.87      0.81       390
       rec.motorcycles       0.69      0.88      0.78       398
       sci.electronics       0.74      0.65      0.69       393
               sci.med       0.76      0.70      0.73       396
             sci.space       0.89      0.75      0.82       394
soc.religion.christian       0.82      0.89      0.86       398
    talk.politics.guns       0.66      0.79      0.72       364
    talk.politics.misc       0.88      0.40      0.55       310

              accuracy                           0.76      381

In [180]:
print("Tworzenie pipeline'u - TfidfVectorizer + stemming + stop_words")
pipeline = Pipeline([
  ('vectorizer',
   TfidfVectorizer(
     tokenizer=TheTokenizer(use_stemming=True, use_stopword_removal=True),
     max_features=1000)
   ),
  ('clf', MultinomialNB()),
])

print("Uczenie pipeline'u")
pipeline.fit(dataset_train.data, dataset_train.target)

print("Ile różnych słów tworzy wektor bag of words (jaki jest rozmiar słownika)")
print(f"W słowniku znajduje się {len(pipeline.named_steps['vectorizer'].vocabulary_.keys())} różnych słów")

print("Ocena klasyfikatora TfidfVectorizer + stemming + stop_words:")
print(
  classification_report(
    labels_as_strings(dataset_test.target),
    labels_as_strings(pipeline.predict(dataset_test.data)))
)

Tworzenie pipeline'u - TfidfVectorizer + stemming + stop_words
Uczenie pipeline'u
Ile różnych słów tworzy wektor bag of words (jaki jest rozmiar słownika)
W słowniku znajduje się 1000 różnych słów
Ocena klasyfikatora TfidfVectorizer + stemming + stop_words:
                        precision    recall  f1-score   support

         comp.graphics       0.68      0.79      0.73       389
 comp.sys.mac.hardware       0.79      0.76      0.77       385
          misc.forsale       0.76      0.87      0.81       390
       rec.motorcycles       0.70      0.90      0.79       398
       sci.electronics       0.75      0.65      0.70       393
               sci.med       0.80      0.72      0.76       396
             sci.space       0.90      0.79      0.84       394
soc.religion.christian       0.88      0.91      0.90       398
    talk.politics.guns       0.68      0.81      0.74       364
    talk.politics.misc       0.86      0.43      0.57       310

              accuracy              

In [181]:
print("Tworzenie pipeline'u - TfidfVectorizer + lemmatization")
pipeline = Pipeline([
  ('vectorizer', TfidfVectorizer(
    tokenizer=TheTokenizer(use_lemmatization=True),
    max_features=1000)
   ),
  ('clf', MultinomialNB()),
])

print("Uczenie pipeline'u")
pipeline.fit(dataset_train.data, dataset_train.target)

print("Ile różnych słów tworzy wektor bag of words (jaki jest rozmiar słownika)")
print(f"W słowniku znajduje się {len(pipeline.named_steps['vectorizer'].vocabulary_.keys())} różnych słów")

print("Ocena klasyfikatora TfidfVectorizer + lemantization:")
print(
  classification_report(
    labels_as_strings(dataset_test.target),
    labels_as_strings(pipeline.predict(dataset_test.data)))
)

Tworzenie pipeline'u - TfidfVectorizer + lemmatization
Uczenie pipeline'u
Ile różnych słów tworzy wektor bag of words (jaki jest rozmiar słownika)
W słowniku znajduje się 1000 różnych słów
Ocena klasyfikatora TfidfVectorizer + lemantization:
                        precision    recall  f1-score   support

         comp.graphics       0.70      0.77      0.73       389
 comp.sys.mac.hardware       0.79      0.76      0.77       385
          misc.forsale       0.77      0.87      0.82       390
       rec.motorcycles       0.68      0.88      0.77       398
       sci.electronics       0.73      0.65      0.69       393
               sci.med       0.78      0.70      0.74       396
             sci.space       0.88      0.77      0.82       394
soc.religion.christian       0.83      0.91      0.87       398
    talk.politics.guns       0.67      0.78      0.72       364
    talk.politics.misc       0.83      0.38      0.52       310

              accuracy                           0.7

In [178]:
print("Tworzenie pipeline'u - TfidfVectorizer + lemmatization + stop_words")
pipeline = Pipeline([
  ('vectorizer',
   TfidfVectorizer(
     tokenizer=TheTokenizer(use_lemmatization=True, use_stopword_removal=True),
     max_features=1000)
   ),
  ('clf', MultinomialNB()),
])

print("Uczenie pipeline'u")
pipeline.fit(dataset_train.data, dataset_train.target)

print("Ile różnych słów tworzy wektor bag of words (jaki jest rozmiar słownika)")
print(f"W słowniku znajduje się {len(pipeline.named_steps['vectorizer'].vocabulary_.keys())} różnych słów")

print("Ocena klasyfikatora TfidfVectorizer + lemmantization + stop_words:")
print(
  classification_report(
    labels_as_strings(dataset_test.target),
    labels_as_strings(pipeline.predict(dataset_test.data)))
)

                        precision    recall  f1-score   support

         comp.graphics       0.70      0.68      0.69       389
 comp.sys.mac.hardware       0.67      0.72      0.70       385
          misc.forsale       0.76      0.84      0.80       390
       rec.motorcycles       0.73      0.86      0.79       398
       sci.electronics       0.65      0.67      0.66       393
               sci.med       0.81      0.63      0.71       396
             sci.space       0.85      0.75      0.80       394
soc.religion.christian       0.90      0.88      0.89       398
    talk.politics.guns       0.61      0.63      0.62       364
    talk.politics.misc       0.58      0.55      0.56       310

              accuracy                           0.73      3817
             macro avg       0.73      0.72      0.72      3817
          weighted avg       0.73      0.73      0.73      3817

Tworzenie pipeline'u - TfidfVectorizer + lemmatization + stop_words
Uczenie pipeline'u
Ile różnych sł

KeyboardInterrupt: 

Wnioski: [ Precision | recall | f1-score | supprt ]
 - bez niczego -- 0.74 0.72 0.71 3817
 - same stemowanie -- 0.77 0.75 0.75 3817
 - stemowanie ze stop_words -- 0.78 0.76 0.76 3817
 - sama lematyzacja -- 0.77 0.75 0.75 3817
 - lematyzacja ze stop_words -- 0.73 0.72 0.72 3817

Część B:

In [165]:
print("Tworzenie pipeline'u - CountVectorizer")
pipeline = Pipeline([
  ('vectorizer', CountVectorizer(tokenizer=TheTokenizer(), max_features=1000)),
  ('clf', MultinomialNB()),
])

print("Uczenie pipeline'u")
pipeline.fit(dataset_train.data, dataset_train.target)

print("Ile różnych słów tworzy wektor bag of words (jaki jest rozmiar słownika)")
print(f"W słowniku znajduje się {len(pipeline.named_steps['vectorizer'].vocabulary_.keys())} różnych słów")

print("Ocena klasyfikatora CountVectorizer:")
print(
  classification_report(
    labels_as_strings(dataset_test.target),
    labels_as_strings(pipeline.predict(dataset_test.data)))
)

print("Tworzenie pipeline'u - TfidfVectorizer")
pipeline = Pipeline([
  ('vectorizer', TfidfVectorizer(tokenizer=TheTokenizer(), max_features=1000)),
  ('clf', MultinomialNB()),
])

print("Uczenie pipeline'u")
pipeline.fit(dataset_train.data, dataset_train.target)

print("Ile różnych słów tworzy wektor bag of words (jaki jest rozmiar słownika)")
print(f"W słowniku znajduje się {len(pipeline.named_steps['vectorizer'].vocabulary_.keys())} różnych słów")

print("Ocena klasyfikatora - TfidfVectorizer:")
print(
  classification_report(
    labels_as_strings(dataset_test.target),
    labels_as_strings(pipeline.predict(dataset_test.data)))
)

Tworzenie pipeline'u - CountVectorizer
Uczenie pipeline'u
Ile różnych słów tworzy wektor bag of words (jaki jest rozmiar słownika)
W słowniku znajduje się 1000 różnych słów
Ocena klasyfikatora CountVectorizer:
                        precision    recall  f1-score   support

         comp.graphics       0.67      0.66      0.66       389
 comp.sys.mac.hardware       0.62      0.66      0.64       385
          misc.forsale       0.72      0.86      0.78       390
       rec.motorcycles       0.61      0.81      0.69       398
       sci.electronics       0.61      0.58      0.59       393
               sci.med       0.78      0.54      0.64       396
             sci.space       0.76      0.69      0.72       394
soc.religion.christian       0.88      0.83      0.86       398
    talk.politics.guns       0.58      0.49      0.53       364
    talk.politics.misc       0.49      0.54      0.51       310

              accuracy                           0.67      3817
             macro a

Lepsze wyniki uzyskuje wektoryzator oparty o Tfid,
pierwszy otrzymał średnią precyzję
 - CountVectorizer - 67%,
 - TfidVectorizer - 74%.